# Ideal gas equation of state using grand canonical ensemble transition-matrix Monte Carlo

In this example, the ideal gas equation of state is obtained as a test of the flat histogram method.

In [1]:
import feasst as fst

monte_carlo = fst.MonteCarlo()
monte_carlo.add(fst.Configuration(fst.MakeDomain(fst.args({"cubic_box_length": "8"})),
                                  fst.args({"particle_type": fst.install_dir()+'/forcefield/data.atom'})))
monte_carlo.add(fst.MakePotential(fst.MakeDontVisitModel()))
monte_carlo.set(fst.MakeThermoParams(fst.args({"beta": str(1./1.2), "chemical_potential": "-3"})))
criteria = fst.MakeFlatHistogram(
    fst.MakeMacrostateNumParticles(fst.Histogram(fst.args({"width": "1", "min": "0", "max": "50"}))),
    fst.MakeTransitionMatrix(fst.args({"min_sweeps": "100"})))
monte_carlo.set(criteria)
monte_carlo.add(fst.MakeTrialTransfer(fst.args({"particle_type": "0"})))
monte_carlo.add(fst.MakeCriteriaUpdater(fst.args({"steps_per": str(1e5)})))
monte_carlo.add(fst.MakeCriteriaWriter(fst.args({"steps_per": str(1e5), "file_name": "id_fh.txt"})))

In [2]:
%time
monte_carlo.run_until_complete()

CPU times: user 13.7 s, sys: 32.6 ms, total: 13.7 s
Wall time: 13.7 s


Check the ideal gas relationship, $\beta P = \rho$

In [3]:
import numpy as np

volume = monte_carlo.configuration().domain().volume()
gce = fst.GrandCanonicalEnsemble(criteria, monte_carlo.thermo_params().beta_mu())
print('rho beta*P difference')
for delta_conjugate in np.arange(-5, 5, 0.1):
        lnpirw = gce.reweight(delta_conjugate)
        if lnpirw.value(lnpirw.size()-1) < -6:
            betaPV = gce.betaPV()
            N = gce.average_macrostate()
            print(N, betaPV, np.abs(1 - betaPV/N))
            assert(np.abs(1 - betaPV/N) < 1e-2)

rho beta*P difference
0.28352371179938896 0.2834109250174366 0.00039780370127273645
0.3133659767306003 0.3132305130399782 0.0004322858915171146
0.3463511041451104 0.3461888474487959 0.00046847460387045015
0.38281022254492336 0.3826164949224828 0.0005060670040436532
0.4231092809374098 0.4228788472982002 0.0005446196753213783
0.46765266421644996 0.46737978153695275 0.0005835157166365068
0.5168871656966174 0.516565698838592 0.0006219284968937977
0.5713063473665461 0.570929980132204 0.0006587835687051902
0.631455320931917 0.6310178983430001 0.000692721360351034
0.6979359862846308 0.697432031310502 0.0007220647509688094
0.771412769454396 0.7708382231478559 0.0007447975056810163
0.8526189105229772 0.85197214642801 0.0007585617524838995
0.9423633649624151 0.9416465232340795 0.0007606850552431821
1.0415384013082887 1.0407590703673615 0.0007482498388424741
1.151128006056329 1.1503012436171116 0.0007182193768786593
1.2722172448799705 1.2713678689055983 0.0006676343822491493
1.4060027781173632 1.

Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!